In [1]:
import jax
import jax.numpy as jnp
import inference.mlp as mlp

In [5]:
c = 1
h = 60
w = 80
sizes = [512, 256, 128]

key = jax.random.PRNGKey(42)

weights = mlp.qs_mlp(c, h, w, sizes, key, 1)

In [6]:
mlp.mlp_serialize_binary(weights, 'weights/weights2.bin')

In [8]:
#tell me how big in bytes that file is in bytes
import os
print(os.path.getsize('weights/weights2.bin'))


20320296
